<a href="https://colab.research.google.com/github/Mametecho/Machine-Translation/blob/main/English_Qafaraf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Necessary library

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import random
import string
import re
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import TextVectorization
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tokenizers import Tokenizer, models, pre_tokenizers, trainers
from sklearn.model_selection import train_test_split
import tensorflow as tf
#from tensorflow.keras import layers, models
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


Import Dataset

In [3]:
usecols = ['English', 'Afar']
df = pd.read_csv("/content/drive/MyDrive/English_afaraf NMT/Dataset/English_afarafih.csv",encoding='latin1', delimiter=',',usecols=usecols)
df.dropna(axis = 0, inplace = True)
df.rename(columns = {"English" : "english_sent", "Afar" : "afar_sent"}, inplace = True)
df.drop_duplicates(inplace=True)
df.head(10)

,english_sent,afar_sent
0,I heard the rusting of palm leaves carried by...,Dananal yeerren qungah farracto oobbe
1,I felt sad,Anu nadaamiteh
2,patriots sacrifice themselves for their country,Agat kacannoytit isinni baaxoh fida yakken
3,Books are exported in paper sacks,Buukat baaxok baaxo kafaqat ruuban
4,"When I heard a dress rusting, I knew my mother...",Sari hargufto oobbe waqdi yiina temeetem eexegeh
5,The man in the accident was safe,Qawwalayla edde tekke num nagay raaqe
6,I rue coming here,Anu akke abe mamaatat nadaamitah
7,Rude children are badly bahaved,adbisinna le urri gexgexit umaane le
8,footballers have to obey rules,koqsoh digira mari le madqah amrisimam faxximta
9,the president rules his country well,ummuunoh abba isi baaxo nagay xiinisa


In [4]:
df.shape

(32451, 2)

Data Preprocessing

In [5]:
contractions = {
        "i'm": "i am",
        "he's": "he is",
        "she's": "she is",
        "it's": "it is",
        "that's": "that is",
        "what's": "what is",
        "where's": "where is",
        "how's": "how is",
        "'ll": " will",
        "'ve": " have",
        "'re": " are",
        "'d": " would",
        "won't": "will not",
        "can't": "cannot",
        "n't": " not",
        "n'": "ng",
        "'bout": "about"
    }
def expand_contractions(text):
        for contraction, expanded in contractions.items():
            text = re.sub(r"\b{}\b".format(contraction), expanded, text)
        return text

In [6]:
def preprocess_text(df):
    # Lowercase the characters
    df["english_sent"] = df["english_sent"].apply(lambda x : x.lower())
    df["afar_sent"] = df["afar_sent"].apply(lambda x : x.lower())


    # Removing digits
    remove_digits = str.maketrans("", "",string.digits)
    df["english_sent"] = df["english_sent"].apply(lambda x : x.translate(remove_digits))
    df["afar_sent"] = df["afar_sent"].apply(lambda x : x.translate(remove_digits))


    # Remove special characters
    special = set(string.punctuation)
    df['english_sent'] = df['english_sent'].apply(lambda x : ''.join(ch for ch in x if ch not in special))
    df['afar_sent'] = df['afar_sent'].apply(lambda x : ''.join(ch for ch in x if ch not in special))

    # Remove quotes
    df['english_sent'] = df['english_sent'].apply(lambda x: re.sub("'", '', x))
    df['afar_sent'] = df['afar_sent'].apply(lambda x: re.sub("'", '', x))

    # Remove extra spaces
    df['english_sent'] = df['english_sent'].apply(lambda x : x.strip())
    df['afar_sent'] = df['afar_sent'].apply(lambda x : x.strip())
    df['english_sent'] = df['english_sent'].apply(lambda x : re.sub(" +"," ",x))
    df['afar_sent'] = df['afar_sent'].apply(lambda x : re.sub(" +"," ",x))


    # Add [start] and [end] tags
    df["afar_sent"] = df["afar_sent"].apply(lambda x : "[start] " + x + " [end]")
    # Remove contracdiction
    df['english_sent'] = df['english_sent'].apply(expand_contractions)

In [7]:
# Preprocess text
preprocess_text(df)

In [8]:
def decode_sequence(input_sentence):
    afar_vocab = af_vectorization.get_vocabulary()
    afar_index_lookup = dict(zip(range(len(afar_vocab)), afar_vocab))
    max_decoded_sentence_length = 20
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = af_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = afar_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence[8:-5]

In [9]:
# For creating Dataset
def format_dataset(eng, af):
    eng = eng_vectorization(eng)
    af = af_vectorization(af)
    return ({"encoder_inputs" : eng, "decoder_inputs" : af[:, :-1],}, af[:, 1:])


def make_dataset(df):
    dataset = tf.data.Dataset.from_tensor_slices((df["english_sent"].values, df["afar_sent"].values))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

In [10]:
# Drop rows with Null values
df.drop(df[df["english_sent"] == " "].index, inplace = True)
df.drop(df[df["afar_sent"] == "[start]  [end]"].index, inplace = True)

In [11]:
 #Find Sentence Length
df["eng_sent_length"] = df["english_sent"].apply(lambda x : len(x.split(' ')))
df["afar_sent_length"] = df["afar_sent"].apply(lambda x : len(x.split(' ')))

In [12]:
# Get sentences with specific length 20
df = df[df["eng_sent_length"] <= 20]
df = df[df["afar_sent_length"] <= 20]

In [13]:
# Take a sample no larger than the DataFrame
sample_size = min(300000, len(df))
df = df.sample(n=sample_size, random_state=2048)
df = df.reset_index(drop=True)

In [14]:

train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

# Splitting val_data into test_data and val_data
test_data, val_data = train_test_split(val_data, test_size=0.5, random_state=42)

In [15]:
train_data.shape,  val_data.shape, test_data.shape

((25901, 4), (3238, 4), (3238, 4))

Define **Parameter**

In [16]:
# Tranining Hyperparameters
batch_size = 64

# Model Hyperparameters
embed_dim = 128
num_heads = 10
latent_dim = 2048
vocab_size = 20000
sequence_length = 20
dropout = 0.2

Tokenizing Sentences


In [17]:
# Using TextVectorization to create sentence vectors
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


In [18]:
eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length
)

af_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length + 1, standardize=custom_standardization
)

eng_vectorization.adapt(df["english_sent"].values)
af_vectorization.adapt(df["afar_sent"].values)

In [19]:
# Savng parameters and weights of both vectorizer
pickle.dump({'config': eng_vectorization.get_config(),
             'weights': eng_vectorization.get_weights()}
            , open("/content/drive/MyDrive/English_afaraf NMT/eng_vectorizer.pkl", "wb"))

pickle.dump({'config': af_vectorization.get_config(),
             'weights': af_vectorization.get_weights()}
            , open("/content/drive/MyDrive/English_afaraf NMT/afar_vectorizer.pkl", "wb"))

Creating **Dataset**

In [20]:
train_ds = make_dataset(train_data)
val_ds = make_dataset(val_data)

Creating Model

In [21]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embedding = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.position_embedding = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim)

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embedding(inputs)
        embedded_positions = self.position_embedding(positions)
        return embedded_tokens + embedded_positions

In [24]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, dropout, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.dropout = dropout
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.layer_norm1 = layers.LayerNormalization()
        self.layer_norm2 = layers.LayerNormalization()
        self.layer_ffn = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"),
             layers.Dropout(dropout),
             layers.Dense(embed_dim),]
        )
        self.supports_masking = True

    def call(self, inputs, mask=None):
        padding_mask = None
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        ffn_input = self.layer_norm1(inputs + attention_output)
        ffn_output = self.layer_ffn(ffn_input)
        return self.layer_norm2(ffn_input + ffn_output)


In [22]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, dropout, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.dropout = dropout
        self.attention1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.layer_ffn = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dropout(dropout),
                layers.Dense(embed_dim),
            ]
        )
        self.layer_norm1 = layers.LayerNormalization()
        self.layer_norm2 = layers.LayerNormalization()
        self.layer_norm3 = layers.LayerNormalization()

        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)

        # Initialize padding_mask
        padding_mask = None

        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = causal_mask

        attention_output1 = self.attention1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out1 = self.layer_norm1(inputs + attention_output1)

        attention_output2 = self.attention2(
            query=out1, value=encoder_outputs, key=encoder_outputs, attention_mask=padding_mask
        )
        out2 = self.layer_norm2(out1 + attention_output2)

        ffn_output = self.layer_ffn(out2)
        return self.layer_norm3(out2 + ffn_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [25]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads, dropout, name="encoder_1")(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads, dropout, name="decoder_1")(x, encoded_seq_inputs)
x = layers.Dropout(0.4)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)


In [24]:
transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 positional_embedding (Posi  (None, None, 128)            2562560   ['encoder_inputs[0][0]']      
 tionalEmbedding)                                                                                 
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                      

In [26]:
# Compile and train the model
transformer.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3)


In [ ]:
# Training model
history = transformer.fit(train_ds, epochs = 10, validation_data = val_ds, callbacks = [early_stopping, reduce_lr])

Epoch 1/10
405/405 [==============================] - 901s 2s/step - loss: 1.9861 - accuracy: 0.7934 - val_loss: 1.3595 - val_accuracy: 0.8235 - lr: 0.0010
Epoch 2/10
405/405 [==============================] - 882s 2s/step - loss: 1.2500 - accuracy: 0.8325 - val_loss: 1.1507 - val_accuracy: 0.8472 - lr: 0.0010
Epoch 3/10
405/405 [==============================] - 908s 2s/step - loss: 1.0216 - accuracy: 0.8536 - val_loss: 1.0379 - val_accuracy: 0.8590 - lr: 0.0010
Epoch 4/10
405/405 [==============================] - 912s 2s/step - loss: 0.8434 - accuracy: 0.8699 - val_loss: 0.9748 - val_accuracy: 0.8653 - lr: 0.0010
Epoch 5/10
405/405 [==============================] - 861s 2s/step - loss: 0.6941 - accuracy: 0.8847 - val_loss: 0.9250 - val_accuracy: 0.8729 - lr: 0.0010
Epoch 6/10
248/405 [=================>............] - ETA: 5:16 - loss: 0.5911 - accuracy: 0.8964

In [27]:
def decode_sequence(input_sentence):
    afar_vocab = af_vectorization.get_vocabulary()
    afar_index_lookup = dict(zip(range(len(afar_vocab)), afar_vocab))
    max_decoded_sentence_length = 20
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = af_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = afar_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence[8:-5]

eng = test_data["english_sent"].values
original = test_data["afar_sent"].values
translated = [decode_sequence(sent) for sent in eng]
bleu = 0
smoothie = SmoothingFunction().method4

for i in range(test_data.shape[0]):
    bleu += sentence_bleu([original[i].split()], translated[i].split(), weights=(0.5, 0.5), smoothing_function=smoothie)

print("BLEU score is:", bleu / test_data.shape[0])

BLEU score is: 0.19627091742719796


In [28]:
# Back-translation
def back_translate(df, model, eng_vectorization, af_vectorization):
    back_translations = []
    for i in range(len(df)):
        english_sentence = df.iloc[i]["english_sent"]
        afar_sentence = decode_sequence(english_sentence, model, eng_vectorization, af_vectorization)
        back_translations.append((english_sentence, afar_sentence))

    back_translated_df = pd.DataFrame(back_translations, columns=["english_sent", "afar_sent"])
    return back_translated_df

def decode_sequence(input_sentence, model, eng_vectorization, af_vectorization):
    afar_vocab = af_vectorization.get_vocabulary()
    afar_index_lookup = dict(zip(range(len(afar_vocab)), afar_vocab))
    max_decoded_sentence_length = 20
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = af_vectorization([decoded_sentence])[:, :-1]
        predictions = model([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = afar_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence[8:-5]


In [29]:
# Create a back-translated dataset
back_translated_df = back_translate(df, transformer, eng_vectorization, af_vectorization)

In [ ]:
# Combine original and back-translated datasets
combined_df = pd.concat([df, back_translated_df])
train_data, val_data = train_test_split(combined_df, test_size=0.2, random_state=42)
test_data, val_data = train_test_split(val_data, test_size=0.5, random_state=42)

NameError: name 'pd' is not defined

In [ ]:
train_data.shape, val_data.shape, test_data.shape

((51803, 4), (6476, 4), (6475, 4))

In [ ]:
train_ds = make_dataset(train_data)
val_ds = make_dataset(val_data)

In [ ]:
transformer.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history = transformer.fit(train_ds, epochs=15, validation_data=val_ds, callbacks=[early_stopping, reduce_lr])


Epoch 1/10
810/810 [==============================] - 1679s 2s/step - loss: 0.5073 - accuracy: 0.9140 - val_loss: 0.6736 - val_accuracy: 0.9008 - lr: 0.0010
Epoch 2/10
810/810 [==============================] - 1628s 2s/step - loss: 0.3782 - accuracy: 0.9292 - val_loss: 0.5697 - val_accuracy: 0.9097 - lr: 0.0010
Epoch 3/10
810/810 [==============================] - 1619s 2s/step - loss: 0.3027 - accuracy: 0.9377 - val_loss: 0.4913 - val_accuracy: 0.9185 - lr: 0.0010
Epoch 4/10
810/810 [==============================] - 1655s 2s/step - loss: 0.2538 - accuracy: 0.9439 - val_loss: 0.4369 - val_accuracy: 0.9277 - lr: 0.0010
Epoch 5/10
810/810 [==============================] - 1625s 2s/step - loss: 0.2219 - accuracy: 0.9486 - val_loss: 0.4091 - val_accuracy: 0.9342 - lr: 0.0010
Epoch 6/10
810/810 [==============================] - 1619s 2s/step - loss: 0.1987 - accuracy: 0.9527 - val_loss: 0.3967 - val_accuracy: 0.9379 - lr: 0.0010
Epoch 7/10
810/810 [==============================] - 1628

In [ ]:
# Saving weights of model
transformer.save_weights("/content/drive/MyDrive/English_afaraf NMT/eng-af.weights.h5")

Testing Model & Calculating BLEU Score

In [ ]:
def decode_sequence(input_sentence):
    afar_vocab = af_vectorization.get_vocabulary()
    afar_index_lookup = dict(zip(range(len(afar_vocab)), afar_vocab))
    max_decoded_sentence_length = 20
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = af_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = afar_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence[8:-5]

eng = test_data["english_sent"].values
original = test_data["afar_sent"].values
translated = [decode_sequence(sent) for sent in eng]
bleu = 0
smoothie = SmoothingFunction().method4

for i in range(test_data.shape[0]):
    bleu += sentence_bleu([original[i].split()], translated[i].split(), weights=(0.5, 0.5), smoothing_function=smoothie)

print("BLEU score is:", bleu / test_data.shape[0])

BLEU score is: 0.4476192838629354


In [ ]:
input_sentence = "she is married while i am a bachelor."
translated_sentence = decode_sequence(input_sentence)
print(f"English: {input_sentence}")
print(f"Translated to Afar: {translated_sentence}")

NameError: name 'decode_sequence' is not defined